# Capstone Project: Amazon Review Classification (Part 1)
Author: **Steven Lee**

# Categorizing Amazon Reviews

User reviews on products and services can often provide potentially valuable feedback to sellers and service providers on various business related areas.  At the very least, for instance, the reviews could signal potential problems with the manufacture of goods, a dip in the quality of services, or some issue with deliveries.  Additionally, they could also provide business owners with many ideas on how to improve products and services.  Better than that, they could even sometimes provide them with ideas of new products or services that already have a demand.

The goal is to build a classification model to categorize reviews into meaningful multi-classes, and help inform on the multiple product aspects that customers find below par, meet expectations or lacking in certain regards.  This new model would have an Accuracy score above 85%.  Models included for comparison will include, Naive Bayes, Random Forest and Neural Networks.

Sentiment analysis merely attempts to see if a review is positive or negative.  While this is helpful, it only tells business owners the proportion of buyers who were happy or unsatisfied with their purchases.  This model will help the business owner gain more meaningful insights about their products and more.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Data" data-toc-modified-id="Import-Data-1">Import Data</a></span></li><li><span><a href="#Check-for-Nulls" data-toc-modified-id="Check-for-Nulls-2">Check for Nulls</a></span></li><li><span><a href="#Fix-Data-Errors" data-toc-modified-id="Fix-Data-Errors-3">Fix Data Errors</a></span></li><li><span><a href="#Impute-Missing-Discrete-Values" data-toc-modified-id="Impute-Missing-Discrete-Values-4">Impute Missing Discrete Values</a></span></li><li><span><a href="#Impute-Missing-Continuous-Values" data-toc-modified-id="Impute-Missing-Continuous-Values-5">Impute Missing Continuous Values</a></span></li><li><span><a href="#Impute-Missing-Ordinal-Values" data-toc-modified-id="Impute-Missing-Ordinal-Values-6">Impute Missing Ordinal Values</a></span></li><li><span><a href="#Impute-Missing-Nominal-Values" data-toc-modified-id="Impute-Missing-Nominal-Values-7">Impute Missing Nominal Values</a></span></li><li><span><a href="#Check-Discrete-Value-Errors" data-toc-modified-id="Check-Discrete-Value-Errors-8">Check Discrete Value Errors</a></span></li><li><span><a href="#Check-Continuous-Value-Errors" data-toc-modified-id="Check-Continuous-Value-Errors-9">Check Continuous Value Errors</a></span></li><li><span><a href="#Impute-Remaining-Ordinal-Values" data-toc-modified-id="Impute-Remaining-Ordinal-Values-10">Impute Remaining Ordinal Values</a></span></li><li><span><a href="#Encode-Remaining-Nominal-Values" data-toc-modified-id="Encode-Remaining-Nominal-Values-11">Encode Remaining Nominal Values</a></span></li><li><span><a href="#Save-Clean-Data-to-File" data-toc-modified-id="Save-Clean-Data-to-File-12">Save Clean Data to File</a></span></li></ul></div>

## Import Data

In [1]:
# Import required libraries.
import numpy as np
import pandas as pd
import gzip
import json

# Set pandas option to display 500 columns.
pd.set_option('display.max_columns', 500)

In [17]:
# Initialise variables.
review_file = "../data/Tools_and_Home_Improvement_5.json.gz"

In [18]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [19]:
%%time

df = getDF(review_file)
df.head()

Wall time: 27.2 s


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"01 28, 2018",AL19QO4XLBQPU,0982085028,{'Style:': ' 1) IR30 POU (30A/3.4kW/110v)'},J. Mollenkamp,"returned, decided against this product",Five Stars,1517097600,NaN,NaN
1,5.0,True,"11 30, 2017",A1I7CVB7X3T81E,0982085028,{'Style:': ' 3) IR260 POU (30A/6kW/220v)'},warfam,Awesome heater for the electrical requirements...,Five Stars,1512000000,NaN,NaN
2,5.0,True,"09 12, 2017",A1AQXO4P5U674E,0982085028,{'Style:': ' Style64'},gbieber2,Keeps the mist of your wood trim and on you. B...,Five Stars,1505174400,NaN,NaN
3,4.0,True,"07 19, 2017",AIRV678P7C4NK,0982085028,NaN,Justin Banner,"So far I hooked it up and tested it , filled a...",it is the perfect temp for a shower,1500422400,NaN,NaN
4,1.0,True,"05 25, 2017",A22I5QDNTNECDW,0982085028,{'Style:': ' 3) IR260 POU (30A/6kW/220v)'},daveparker,"i installed this 10 months ago, instructions w...",worked well...for 10 months.,1495670400,16,NaN


In [20]:
df.shape

(2070831, 12)

In [21]:
df.dtypes

overall           float64
verified             bool
reviewTime         object
reviewerID         object
asin               object
style              object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
vote               object
image              object
dtype: object

In [22]:
for i in range(10):
    print(df.loc[i, 'reviewText'])
    input("...\n")

returned, decided against this product
...

Awesome heater for the electrical requirements! Makes an awesome preheater for my talnkless system
...

Keeps the mist of your wood trim and on you. Bendable too.
...

So far I hooked it up and tested it , filled a five gallon bucket with hot water, it is the perfect temp for a shower,the flow valve that came with it broke when i tried to tighten it to get it to stop leaking,just hooked it directly to sharkbite pex fitting,seems to work fine without the valve. it is hooked up to a 2 poll 20 amp breaker, will have to wait till the rest of my bathroom is finished to give a better review.
...

i installed this 10 months ago, instructions were easy and it worked well. this week, the water heater is stuck on 26/27. i tried all the different modes from 1-4, reset my circuit breakers, no changes. it won't heat water anymore.

UPDATE 1-20-18

Their warranty requires you to have a licensed installer. But that isn't my complaint. Its their useless serv

In [ ]:
# Read in training dataset and view the first 5 records.
data = pd.read_csv("../data/train.csv")
data.head()

In [ ]:
# Read in test dataset and view the first 5 records.
df = pd.read_csv("../data/test.csv")
df.head()

In [ ]:
# Check number of records and columns.
data.shape

In [ ]:
# Check for data types and nulls.
data.info()

## Check for Nulls

In [ ]:
data.isnull().sum()

## Fix Data Errors

## Impute Missing Discrete Values

In [ ]:
# Check discrete variables for nulls.
discrete_vars = [
    'Id',
    'Yr Sold'
]

ames_housing[discrete_vars].isnull().sum()

In [ ]:
# Explore discrete Bsmt variables with nulls.
bsmt_vars = [
    'Bsmt Qual',
    'Bsmt Half Bath'
]

ames_housing.loc[(ames_housing['Bsmt Full Bath'].isnull()) | (ames_housing['Bsmt Half Bath'].isnull()), bsmt_vars]

# The 2 records (index 616 and 1327) are the same for both 'Bsmt Full Bath' is null and 'Bsmt Half Bath' is null.

In [ ]:
# Total Bsmt SF has 1 null record and is of float data type, so check also for zero values.
ames_housing.loc[(ames_housing['Total Bsmt SF'].isnull()) | (ames_housing['Total Bsmt SF'] < 1), bsmt_vars].head()

# It is evident that all these 114 houses have no basement.

In [ ]:
# Update the 2 records to be consistent with the other records without basement.
ames_housing.at[616, 'Bsmt Full Bath'] = 0

In [ ]:
# Explore discrete Garage variables with nulls.
garage_vars = [
    'Garage Type',
    'Garage Cond'
]

ames_housing.loc[(ames_housing['Garage Yr Blt'].isnull()) | (ames_housing['Garage Cars'].isnull()), garage_vars].head()

In [ ]:
# Check for record with null Garage Cars.
ames_housing.loc[(ames_housing['Garage Cars'].isnull()), garage_vars]

# Null Garage Cars record also has null Garage Yr Blt, but has a valid Garage Type.

In [ ]:
# Garage Area is of float data type, so check also for zero values.
ames_housing.loc[(ames_housing['Garage Area'] < 1), garage_vars].count()

# Except for the 1 record with a non-null Garage Type and null Garage Yr Blt, 
# it is evident the rest of the records have no garage.

In [ ]:
# Inpute Garage Yr Blt null values to the Year Built (Original construction date).
ames_housing['Garage Yr Blt'] = ames_housing['Garage Yr Blt'].fillna(ames_housing['Year Built'])

In [ ]:
# Assuming that the record with null Garage Cars has an error Garage Type, 
# since all other garage related columns have null values.
# Updating this record to be consistent with other records without a garage.
ames_housing.at[1712, 'Garage Type'] = np.nan
ames_housing.at[1712, 'Garage Cars'] = 0

## Impute Missing Continuous Values

In [ ]:
# Check continuous variables for nulls.
continuous_vars = [
    'Lot Frontage',
    'Misc Val'
]

ames_housing[continuous_vars].isnull().sum()

In [ ]:
# Explore continuous Lot Frontages with nulls.
lot_vars = [
    'Lot Frontage',
    'Lot Area',
    'Lot Shape',
    'Lot Config'
]

# Check if Lot Area values (has no null values) are valid.
ames_housing.loc[ames_housing['Lot Area'] < 1, lot_vars]

In [ ]:
ames_housing.loc[ames_housing['Lot Area'] <= 27*ames_housing['Lot Frontage'], lot_vars]

# Lot Frontage is less than 5% the size of the Lot Area.

In [ ]:
# Check allocation of Lot Config values.
ames_housing['Lot Config'].value_counts()

In [ ]:
# Impute null Lot Frontage values to mean values for respective Lot Configs.
for config in ames_housing['Lot Config'].unique():
    # Find mean for respective Lot Config.
    mean_frontage = round(ames_housing['Lot Frontage'][(ames_housing['Lot Config'] == config) & 
                                                       (~ames_housing['Lot Frontage'].isnull())].mean())
    print(f"Mean Lot Frontage for '{config}' Lot Config is: {mean_frontage}.")
    # Impute null Lot Frontage to mean for respective Lot Config.
    ames_housing.loc[(ames_housing['Lot Frontage'].isnull()) & (ames_housing['Lot Config'] == config), 
                     'Lot Frontage'] = mean_frontage

In [ ]:
# Explore continuous Mas Vnr Area with nulls.
mas_vnr_vars = [
    'Mas Vnr Type',
    'Mas Vnr Area'
]

# Check if null Mas Vnr Area values also have null Mas Vnr Type values.
ames_housing.loc[(ames_housing['Mas Vnr Area'].isnull()) & (ames_housing['Mas Vnr Type'].isnull()), 
                 mas_vnr_vars].isnull().sum()

# These 22 houses are without any masonry veneers.

In [ ]:
# Check allocation of Mas Vnr Types.
ames_housing['Mas Vnr Type'].value_counts()

In [ ]:
# Check inconsistencies between type and area values.
ames_housing.loc[((ames_housing['Mas Vnr Type'] == 'None') & (ames_housing['Mas Vnr Area'] > 0)) | 
                 ((ames_housing['Mas Vnr Type'] != 'None') & (ames_housing['Mas Vnr Area'] == 0)), 
                 mas_vnr_vars].value_counts()

# (*) Need to circle back to address these 8 inconsistencises when looking into nominal variables.

In [ ]:
# Update Mas Vnr Area to 0 where Mas Vnr Type is None and Mas Vnr Area is one.
ames_housing.loc[(ames_housing['Mas Vnr Area'] == 1) & (ames_housing['Mas Vnr Type'] == 'None'), 'Mas Vnr Area'] = 0

In [ ]:
# Impute erroneous Mas Vnr Type and Mas Vnr Area values to mean values based on respective Types.

mean_size = round(ames_housing['Mas Vnr Area'][(ames_housing['Mas Vnr Type'] == 'BrkFace') & 
                                               (ames_housing['Mas Vnr Area'] > 0)].mean())
print(f"Mean BrkFace Mas Vnr Area size: {mean_size}.")
ames_housing.loc[(ames_housing['Mas Vnr Area'] == 0) & (ames_housing['Mas Vnr Type'] == 'BrkFace'), 
                 'Mas Vnr Area'] = mean_size

In [ ]:
# Setting the Area size to 0 for the 2 records with None type and size greater than 0, 
# since it is not possible to infer the Mas Vnr Type without more information.
ames_housing.loc[(ames_housing['Mas Vnr Type'] == 'None') & (ames_housing['Mas Vnr Area'] > 0), 'Mas Vnr Area'] = 0

In [ ]:
# Update null values for Mas Vnr Type and Mas Vnr Area.
ames_housing['Mas Vnr Type'] = ames_housing['Mas Vnr Type'].fillna('None')
ames_housing['Mas Vnr Area'] = ames_housing['Mas Vnr Area'].fillna(0)

## Impute Missing Ordinal Values

In [ ]:
# Check ordinal variables for nulls.
ordinal_vars = [
    'Lot Shape', 
    'Fence'
]

ames_housing[ordinal_vars].isnull().sum()

In [ ]:
# Explore ordinal Bsmt related values with nulls.
ames_housing[bsmt_vars].isnull().sum()

In [ ]:
# Check for null values in the above Bsmt related values, but where Total Bsmt SF is greater than zero.
ames_housing.loc[((ames_housing['Bsmt Qual'].isnull()) | (ames_housing['Bsmt Cond'].isnull()) |
                 (ames_housing['BsmtFin Type 1'].isnull()) | (ames_housing['BsmtFin Type 2'].isnull()) |
                 (ames_housing['Bsmt Exposure'].isnull())) & (ames_housing['Total Bsmt SF'] > 0), bsmt_vars]

In [ ]:
# Update null Bsmt Exposure values to No (No Exposure) for the 3 records above as Total Bsmt SF is not zero.
ames_housing.loc[(ames_housing['Bsmt Exposure'].isnull()) & (ames_housing['Total Bsmt SF'] > 0), 
                 'Bsmt Exposure'] = 'No'

# Update the null BsmtFin Type 2 value to NA (No Basement) for the single record above, 
# even though the house has a basement.  For fair computation reasons.
ames_housing.loc[(ames_housing['BsmtFin Type 2'].isnull()) & (ames_housing['Total Bsmt SF'] > 0), 
                 'BsmtFin Type 2'] = 'NA'

In [ ]:
# Check for the remaining 55 null value records, which are houses without any basement i.e. Total Bsmt SF is zero.
ames_housing.loc[((ames_housing['Bsmt Qual'].isnull()) | (ames_housing['Bsmt Cond'].isnull()) |
                 (ames_housing['BsmtFin Type 1'].isnull()) | (ames_housing['BsmtFin Type 2'].isnull()) |
                 ames_housing['Bsmt Exposure'].isnull()) & (ames_housing['Total Bsmt SF'] < 1), bsmt_vars].isnull().sum()

In [ ]:
# Update the 55 records with Bsmt related null values accordingly as houses with No Basement (NA).
ames_housing.loc[(ames_housing['Bsmt Qual'].isnull()) & (ames_housing['Total Bsmt SF'] == 0), 'Bsmt Qual'] = 'NA'

In [ ]:
# Map ordinal values to numeric for the 5 basement related columns.
ames_housing['BsmtFin Type 1'] = ames_housing['BsmtFin Type 1'].map({'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 
                                                                     'LwQ': 2, 'Unf': 1, 'NA': 0})

In [ ]:
# Explore ordinal Fireplace Qu with nulls.
fireplace_vars = [
    'Fireplaces', 
    'Fireplace Qu'
]

ames_housing.loc[ames_housing['Fireplaces'] < 1, fireplace_vars].nunique(), 

# These 1000 houses are without fireplaces.

In [ ]:
# Update the 1000 records with Fireplace Qu null values with NA.
ames_housing.loc[ames_housing['Fireplace Qu'].isnull(), 'Fireplace Qu'] = 'NA'

In [ ]:
# Map ordinal values to numeric for Fireplace Qu.
ames_housing['Fireplace Qu'] = ames_housing['Fireplace Qu'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0})

In [ ]:
# Explore ordinal Garage variables with nulls.
ames_housing.loc[((ames_housing['Garage Finish'].isnull()) | (ames_housing['Garage Qual'].isnull()) | 
                 (ames_housing['Garage Cond'].isnull())) & (ames_housing['Garage Area'] < 1), garage_vars].head()

# These 114 records are houses without garages.

In [ ]:
# Update these 114 records with Garage related null values accordingly as houses with No Garage (NA).
ames_housing.loc[(ames_housing['Garage Finish'].isnull()) & (ames_housing['Garage Area'] == 0), 'Garage Finish'] = 'NA'

In [ ]:
# Map ordinal values to numeric for the 3 garage related columns.
ames_housing['Garage Cond'] = ames_housing['Garage Cond'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0})

In [ ]:
# Explore ordinal Pool QC with nulls.
pool_vars = [
    'Pool Area', 
    'Pool QC'
]
ames_housing.loc[ames_housing['Pool Area'] == 0, pool_vars].head()

In [ ]:
ames_housing.loc[ames_housing['Pool Area'] == 0, pool_vars].count(), 

# These 2042 houses are without pools.

In [ ]:
# Update these 2042 records with Pool QC null values as houses with No Pool (NA).
ames_housing.loc[(ames_housing['Pool QC'].isnull()) & (ames_housing['Pool Area'] == 0), 'Pool QC'] = 'NA'

In [ ]:
# Map ordinal values to numeric for the Pool QC column.
ames_housing['Pool QC'] = ames_housing['Pool QC'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0})

In [ ]:
# Explore ordinal Fence with nulls.
ames_housing['Fence'].value_counts()

# Will assume that these 1651 houses have no fence or no fence rating.

In [ ]:
# Update these 1651 records with Fence null values as houses with No Fence (NA).
ames_housing.loc[(ames_housing['Fence'].isnull()), 'Fence'] = 'NA'

In [ ]:
# Map ordinal values to numeric for Fence column.
ames_housing['Fence'] = ames_housing['Fence'].map({'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'NA': 0})

## Impute Missing Nominal Values

In [ ]:
# Check nominal variables for nulls.
nominal_vars = [
    'PID', 
    'Sale Type'
]

ames_housing[nominal_vars].isnull().sum()

In [ ]:
# Explore nominal Alley with nulls.
pave_vars = [
    'Street', 
    'Alley', 
    'Paved Drive'
]

ames_housing.loc[~ames_housing['Alley'].isnull(), pave_vars].value_counts()

# No noticeable connection between Street, Alley and Paved Drive.

In [ ]:
# Update these 1911 records with null Alley values as houses with No alley access (NA).
ames_housing.loc[ames_housing['Alley'].isnull(), 'Alley'] = 'NA'

# Dummy column creation on Alley to get new Alley Grvl, Alley Pave and Alley NA columns.
ames_housing = pd.get_dummies(ames_housing, columns=['Alley'], prefix='Alley', prefix_sep=' ')

# Drop reference category column Alley NA.
ames_housing.drop('Alley NA', axis=1, inplace=True)

In [ ]:
# Explore nominal Garage Type with nulls.
ames_housing.loc[ames_housing['Garage Type'].isnull(), garage_vars].count()

# These 114 records have been identified before to be houses without garages.

In [ ]:
# Update these 114 records with null Garage Type values as houses with No Garage (NA).
ames_housing.loc[ames_housing['Garage Type'].isnull(), 'Garage Type'] = 'NA'

# Dummy column creation on Garage Type to get 7 new columns prefixed with 'GTy'.
ames_housing = pd.get_dummies(ames_housing, columns=['Garage Type'], prefix='GTy', prefix_sep=' ')

# Drop reference category column GTy NA.
ames_housing.drop('GTy NA', axis=1, inplace=True)

In [ ]:
# Explore nominal Alley with nulls.
misc_vars = [
    'Misc Feature', 
    'Misc Val'
]

ames_housing.loc[(ames_housing['Misc Feature'].isnull()) | (ames_housing['Misc Val'] == 0), misc_vars].count()

# These 1986 houses with null Misc Feature values, also have zero Misc Val.

In [ ]:
ames_housing.loc[(ames_housing['Misc Feature'].isnull()) | (ames_housing['Misc Val'] < 1), misc_vars].head()

# These 1986 houses have no miscellaneous features.

In [ ]:
# Update these 1986 records with null Misc Feature values as houses with None (NA).
ames_housing.loc[ames_housing['Misc Feature'].isnull(), 'Misc Feature'] = 'NA'

# Dummy column creation on Garage Type to get 6 new columns prefixed with 'Misc'.
ames_housing = pd.get_dummies(ames_housing, columns=['Misc Feature'], prefix='Misc', prefix_sep=' ')

# Drop reference category column Misc NA.
ames_housing.drop('Misc NA', axis=1, inplace=True)

## Check Discrete Value Errors

In [ ]:
ames_housing[discrete_vars].describe()

In [ ]:
# Check that Id is unique for every record, and it is unique.
ames_housing['Id'].nunique()

In [ ]:
# Error spotted where the max Garagae Yr Blt is 2207.
ames_housing.loc[ames_housing['Garage Yr Blt'] > 2010, discrete_vars]

In [ ]:
# Correct Garage Yr Blt to be same as Year Remod/Add and Yr Sold.
ames_housing.loc[ames_housing['Garage Yr Blt'].isnull(), 'Garage Yr Blt'] = 2007

## Check Continuous Value Errors

In [ ]:
ames_housing[continuous_vars].describe()

In [ ]:
# Check that the total basement size add up.
ames_housing.loc[(ames_housing['BsmtFin SF 1'] + ames_housing['BsmtFin SF 2'] + ames_housing['Bsmt Unf SF'])
                > ames_housing['Total Bsmt SF'], continuous_vars]

# No errors found.

## Impute Remaining Ordinal Values

In [ ]:
# Map ordinal values to numeric ones.
ames_housing['Functional'] = ames_housing['Functional'].map({'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 
                                                             'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal': 1})

## Encode Remaining Nominal Values

In [ ]:
# Create dummy columns to encode nominal values and drop reference category column.
data = pd.get_dummies(ames_housing, columns=['MS SubClass'], prefix='MSSubCl', prefix_sep=' ', drop_first=True)
data = pd.get_dummies(ames_housing, columns=['Sale Type'], prefix='SaleTy', prefix_sep=' ')
data.drop('SaleTy Oth', axis=1, inplace=True)

## Save Clean Data to File

In [ ]:
# Save clean training data to file.
data.to_csv("../data/train_clean.csv", index=False)

In [ ]:
# Save clean test data to file.
data.to_csv("../data/test_clean.csv", index=False)